In [1]:
import os, io
from getpass import getpass
from BiConn import BiConn
from Auxs   import hashs
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.exceptions import *
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, hmac, cmac
from cryptography.hazmat.primitives.asymmetric import dh,dsa
from cryptography.hazmat.primitives import serialization,hashes

# Generate some parameters DH
parameters_dh = dh.generate_parameters(generator=2, key_size=1024,
                                     backend=default_backend())

# Generate some parameters DSA
parameters_dsa = dsa.generate_parameters(key_size=1024,backend=default_backend())

default_algorithm = hashes.SHA256
# seleciona-se um dos vários algorimos implementados na package

def my_mac(key):
#    return hashes.Hash(default_algorithm(),backend=default_backend())
     return hmac.HMAC(key,default_algorithm(),default_backend())
#    return cmac.CMAC(algorithms.AES(key), backend=default_backend())

## Diffie Hellman & DSA

In [18]:
def Dh(conn):
    # agreement
    pk = parameters_dh.generate_private_key()
    pub = pk.public_key().public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo)
    conn.send(pub)

    
    # shared_key calculation
    peer_pub = conn.recv()
    peer_pub_key = serialization.load_pem_public_key(
            peer_pub,
            backend=default_backend())
    shared_key = pk.exchange(peer_pub_key)
    
    # confirmation
    my_tag = hashs(bytes(shared_key))
    conn.send(my_tag)
    peer_tag = conn.recv()
    if my_tag == peer_tag:
        print('OK')
    else:
        print('FAIL')
        
    
    #ASSINAR
    
   #gerar as chaves privada e pública
    private_key_dsa = parameters_dsa.generate_private_key()
    pub_dsa = private_key_dsa.public_key().public_bytes( 
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo)
    
    #envia a chave pública
    conn.send(pub_dsa)
    
    #cálculo da assinatura
    signature = private_key_dsa.sign(pub, hashes.SHA256())
   
    peer_pub_dsa =serialization.load_pem_public_key(
        conn.recv(), 
        backend=default_backend())  
    
    
    conn.send(signature)
    
    
    #verificar
    
    
    try:
        sig = conn.recv()
        peer_pub_dsa.verify(sig, peer_pub, hashes.SHA256())
        print("ok")
        return my_tag
    except InvalidSignature:
        print("fail")

## Emitter

In [19]:
message_size = 2**10

def Emitter(conn):
    # Acordo de chaves DH e assinatura DSA
    key = Dh(conn)
    
    # Mensagem
    inputs = io.BytesIO(bytes('1'*message_size,'utf-8'))
    
    # nonce para a cifra
    nonce  = os.urandom(16)
    
    # Cifra
    cipher = Cipher(algorithms.AES(key), modes.CTR(nonce), 
                        backend=default_backend()).encryptor()
    
    # HMAC
    mac = my_mac(key)
    
    conn.send(nonce) # Envio do nonce
    buffer = bytearray(32) # Buffer onde vão ser lidos os blocos
    
    # lê, cifra e envia sucessivos blocos do input 
    try:     
        while inputs.readinto(buffer): 
            ciphertext = cipher.update(bytes(buffer))
            mac.update(ciphertext)
            conn.send((ciphertext, mac.copy().finalize()))         

        conn.send((cipher.finalize(), mac.finalize()))    # envia a finalização
    except Exception as err:
        print("Erro no emissor: {0}".format(err))

    inputs.close()          # fecha a 'input stream'
    conn.close()            # fecha a conecção

## Receiver

In [20]:
def Receiver(conn):
    # Acordo de chaves DH e assinatura DSA
    key = Dh(conn)
    
    # Inicializa um output stream para receber o texto decifrado
    outputs = io.BytesIO()
    
    # Recebe o nonce
    nonce = conn.recv()
    
    # Cifra
    cipher = Cipher(algorithms.AES(key), modes.CTR(nonce), 
                        backend=default_backend()).decryptor()
    
    # HMAC
    mac = my_mac(key)
    
    # operar a cifra: ler da conecção um bloco, autenticá-lo, decifrá-lo e escrever o resultado no 'stream' de output
    try:
        while True:
            try:
                buffer, tag = conn.recv()
                ciphertext = bytes(buffer)
                mac.update(ciphertext)
                if tag != mac.copy().finalize():
                    raise InvalidSignature("erro no bloco intermédio")
                outputs.write(cipher.update(ciphertext))

            except EOFError:
                if tag != mac.finalize():
                    raise InvalidSignature("erro na finalização")                
                outputs.write(cipher.finalize())
                break
            except InvalidSignature as err:
                raise Exception("autenticação do ciphertext ou metadados: {}".format(err))
        
        print(outputs.getvalue())     # verificar o resultado
        
    except Exception as err:
        print("Erro no receptor: {0}".format(err))
        
    outputs.close()    # fechar 'stream' de output
    conn.close()       # fechar a conecção

In [24]:
BiConn(Emitter, Receiver, timeout=30).auto()

OK
OK
ok
ok


Process Process-22:


KeyboardInterrupt: 

Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-20-f0096520669e>", line 22, in Receiver
    buffer, tag = conn.recv()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
